# 📊 TELECOM X - CUSTOMER CHURN ANALYSIS
## ⭐ MASTER NOTEBOOK - Complete Analysis Pipeline

---

### 🎯 **This Notebook Includes:**

1. ⚙️ **Setup & Configuration**
2. 🔄 **ETL Pipeline** (Extract, Transform, Load)
3. 🔍 **Exploratory Data Analysis (EDA)**
4. 📈 **Data Visualization**
5. 📐 **Statistical Analysis**
6. 🤖 **Machine Learning Preview**
7. 📄 **Report Generation** (CSV, Excel, PDF)

---

### 🌍 **Multi-Language Support:**
🇪🇸 Español | 🇺🇸 English | 🇧🇷 Português | 🇫🇷 Français | 🇸🇦 العربية | 🇮🇱 עברית | 🇨🇳 中文

---

### 👤 **Developed by:**
**Elizabeth Díaz Familia**

🔗 [GitHub](https://github.com/Lizzy0981) | 🔗 [LinkedIn](https://linkedin.com/in/eli-familia/) | 🔗 [Twitter](https://twitter.com/Lizzyfamilia)

---

### 📝 **How to Use:**

1. Click **Runtime** → **Run all** (or Ctrl+F9)
2. Wait for all cells to execute
3. Check the `reports/` folder for generated files

---

## 🎯 SECTION 1: SETUP & DEPENDENCIES

In [ ]:
%%capture
# Install dependencies (silent)
!pip install -q pandas numpy plotly matplotlib seaborn scikit-learn scipy
!pip install -q openpyxl reportlab requests tqdm

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import requests
import json
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print('═' * 70)
print('📊 TELECOM X - CHURN ANALYSIS MASTER NOTEBOOK')
print('═' * 70)
print(f'✅ All libraries loaded successfully')
print(f'🐍 Pandas: {pd.__version__}')
print(f'🔢 NumPy: {np.__version__}')
print(f'📊 Plotly: {px.__version__}')
print('═' * 70)

In [ ]:
# Create directory structure
directories = [
    'data/raw', 'data/processed', 'data/api_data',
    'reports/csv', 'reports/excel', 'reports/pdf', 'reports/temp',
    'visualizations/plots', 'visualizations/interactive',
    'config', 'translations'
]

for directory in directories:
    Path(directory).mkdir(parents=True, exist_ok=True)

print('✅ Directory structure created')

## 🔄 SECTION 2: ETL PIPELINE

In [ ]:
print('\n' + '═' * 70)
print('🔄 ETL PIPELINE - DATA EXTRACTION')
print('═' * 70)

# Generate mock telecom data
def generate_telecom_data(n=1000):
    np.random.seed(42)
    data = {
        'CustomerID': [f'CUST{i:05d}' for i in range(1, n+1)],
        'Gender': np.random.choice(['Male', 'Female'], n),
        'SeniorCitizen': np.random.choice([0, 1], n, p=[0.84, 0.16]),
        'Partner': np.random.choice(['Yes', 'No'], n, p=[0.48, 0.52]),
        'Dependents': np.random.choice(['Yes', 'No'], n, p=[0.30, 0.70]),
        'tenure': np.random.randint(1, 73, n),
        'PhoneService': np.random.choice(['Yes', 'No'], n, p=[0.90, 0.10]),
        'InternetService': np.random.choice(['DSL', 'Fiber optic', 'No'], n, p=[0.34, 0.44, 0.22]),
        'Contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], n, p=[0.55, 0.21, 0.24]),
        'PaperlessBilling': np.random.choice(['Yes', 'No'], n, p=[0.59, 0.41]),
        'PaymentMethod': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer', 'Credit card'], n),
        'MonthlyCharges': np.random.uniform(18.0, 120.0, n),
    }
    
    df = pd.DataFrame(data)
    df['TotalCharges'] = df['tenure'] * df['MonthlyCharges'] + np.random.uniform(-50, 50, n)
    df['TotalCharges'] = df['TotalCharges'].clip(lower=0).round(2)
    df['MonthlyCharges'] = df['MonthlyCharges'].round(2)
    
    # Generate churn with realistic factors
    churn_prob = 0.27
    churn_factors = (
        (df['Contract'] == 'Month-to-month').astype(int) * 0.3 +
        (df['tenure'] < 12).astype(int) * 0.2 +
        (df['InternetService'] == 'Fiber optic').astype(int) * 0.1
    )
    df['Churn'] = (np.random.random(n) < (churn_prob + churn_factors * 0.5)).map({True: 'Yes', False: 'No'})
    
    return df

# Generate data
df = generate_telecom_data(1000)
print(f'✅ Generated {len(df):,} customer records')
print(f'📊 Columns: {len(df.columns)}')

# Display sample
display(df.head())

In [ ]:
print('\n' + '═' * 70)
print('🔧 ETL - DATA TRANSFORMATION')
print('═' * 70)

# Create derived features
df['TenureGroup'] = pd.cut(df['tenure'], bins=[0,12,24,48,73], labels=['0-12 mo', '12-24 mo', '24-48 mo', '48+ mo'])

df['ChargesGroup'] = pd.qcut(df['MonthlyCharges'], q=3, labels=['Low', 'Medium', 'High'])

# Enrich with API data
try:
    response = requests.get('https://api.exchangerate-api.com/v4/latest/USD', timeout=5)
    if response.status_code == 200:
        rate = response.json()['rates'].get('EUR', 0.85)
        df['MonthlyCharges_EUR'] = (df['MonthlyCharges'] * rate).round(2)
        print(f'✅ Exchange rate applied: USD/EUR = {rate}')
except:
    print('⚠️ Could not fetch exchange rates')

# Save processed data
df.to_csv('data/processed/telecom_churn_clean.csv', index=False)
print('✅ Data saved to data/processed/telecom_churn_clean.csv')

## 🔍 SECTION 3: EXPLORATORY DATA ANALYSIS

In [ ]:
print('\n' + '═' * 70)
print('🔍 EXPLORATORY DATA ANALYSIS')
print('═' * 70)

# Basic statistics
print('\n📊 Dataset Overview:')
print(f'Total Records: {len(df):,}')
print(f'Total Features: {len(df.columns)}')
print(f'\n📈 Churn Distribution:')
churn_dist = df['Churn'].value_counts()
for status, count in churn_dist.items():
    pct = (count/len(df))*100
    print(f'  {status}: {count:,} ({pct:.1f}%)')

# Key metrics
print(f'\n💰 Financial Metrics:')
print(f'  Total Revenue: ${df["TotalCharges"].sum():,.2f}')
print(f'  Avg Monthly: ${df["MonthlyCharges"].mean():.2f}')
print(f'  Avg Tenure: {df["tenure"].mean():.1f} months')

# Statistical summary
print('\n📋 Statistical Summary:')
display(df[['tenure', 'MonthlyCharges', 'TotalCharges']].describe())

In [ ]:
# Churn analysis by segments
print('\n📊 Churn Analysis by Segments:')
print('\nBy Contract Type:')
churn_by_contract = df.groupby('Contract')['Churn'].apply(
    lambda x: f"{(x=='Yes').mean():.2%}"
)
print(churn_by_contract)

print('\nBy Internet Service:')
churn_by_internet = df.groupby('InternetService')['Churn'].apply(
    lambda x: f"{(x=='Yes').mean():.2%}"
)
print(churn_by_internet)

## 📈 SECTION 4: DATA VISUALIZATION

In [ ]:
print('\n' + '═' * 70)
print('📈 DATA VISUALIZATION')
print('═' * 70)

# 1. Churn Distribution (Pie Chart)
fig = px.pie(df, names='Churn', title='📊 Churn Distribution',
             color_discrete_sequence=['#00d4aa', '#f5576c'])
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

print('✅ Pie chart displayed')

In [ ]:
# 2. Monthly Charges Distribution
fig = px.histogram(df, x='MonthlyCharges', color='Churn',
                   title='💰 Monthly Charges Distribution by Churn Status',
                   color_discrete_sequence=['#00d4aa', '#f5576c'],
                   barmode='overlay', opacity=0.7)
fig.update_layout(xaxis_title='Monthly Charges ($)', yaxis_title='Count')
fig.show()

print('✅ Histogram displayed')

In [ ]:
# 3. Churn by Contract (Bar Chart)
churn_contract = df.groupby('Contract')['Churn'].apply(
    lambda x: (x=='Yes').mean()*100
).reset_index()
churn_contract.columns = ['Contract', 'ChurnRate']

fig = px.bar(churn_contract, x='Contract', y='ChurnRate',
             title='📋 Churn Rate by Contract Type',
             color='ChurnRate',
             color_continuous_scale='reds',
             text='ChurnRate')
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(yaxis_title='Churn Rate (%)')
fig.show()

print('✅ Bar chart displayed')

## 📐 SECTION 5: STATISTICAL ANALYSIS

In [ ]:
print('\n' + '═' * 70)
print('📐 STATISTICAL ANALYSIS')
print('═' * 70)

# Chi-Square Test: Churn vs Contract
print('\n🔬 Chi-Square Test: Churn vs Contract Type')
contingency = pd.crosstab(df['Contract'], df['Churn'])
chi2, p_value, dof, expected = stats.chi2_contingency(contingency)
print(f'Chi-Square Statistic: {chi2:.4f}')
print(f'P-value: {p_value:.6f}')
print(f'Degrees of Freedom: {dof}')
if p_value < 0.05:
    print('✅ Result: Significant relationship (p < 0.05)')
else:
    print('❌ Result: No significant relationship')

# T-Test: Monthly Charges
print('\n🔬 T-Test: Monthly Charges (Churned vs Active)')
churned = df[df['Churn']=='Yes']['MonthlyCharges']
active = df[df['Churn']=='No']['MonthlyCharges']
t_stat, p_value = stats.ttest_ind(churned, active)
print(f'T-Statistic: {t_stat:.4f}')
print(f'P-value: {p_value:.6f}')
print(f'Mean Churned: ${churned.mean():.2f}')
print(f'Mean Active: ${active.mean():.2f}')
if p_value < 0.05:
    print('✅ Result: Significant difference (p < 0.05)')
else:
    print('❌ Result: No significant difference')

## 🤖 SECTION 6: MACHINE LEARNING PREVIEW

In [ ]:
print('\n' + '═' * 70)
print('🤖 MACHINE LEARNING - CUSTOMER SEGMENTATION')
print('═' * 70)

# Prepare features for clustering
features = df[['tenure', 'MonthlyCharges', 'TotalCharges']].copy()
features = features.fillna(0)

# Scale features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(features_scaled)

print(f'✅ K-Means clustering completed')
print(f'\n📊 Cluster Distribution:')
cluster_dist = df['Cluster'].value_counts().sort_index()
for cluster, count in cluster_dist.items():
    pct = (count/len(df))*100
    print(f'  Cluster {cluster}: {count:,} customers ({pct:.1f}%)')

# Cluster characteristics
print(f'\n📋 Cluster Characteristics:')
cluster_summary = df.groupby('Cluster').agg({
    'tenure': 'mean',
    'MonthlyCharges': 'mean',
    'Churn': lambda x: (x=='Yes').mean()
}).round(2)
cluster_summary.columns = ['Avg Tenure', 'Avg Monthly', 'Churn Rate']
display(cluster_summary)

In [ ]:
# Visualize clusters
fig = px.scatter(df, x='tenure', y='MonthlyCharges',
                 color='Cluster', symbol='Churn',
                 title='🎯 Customer Segments (K-Means Clustering)',
                 labels={'Cluster': 'Segment'},
                 hover_data=['TotalCharges', 'Contract'])
fig.update_traces(marker=dict(size=8))
fig.show()

print('✅ Cluster visualization displayed')

## 📄 SECTION 7: REPORT GENERATION

In [ ]:
print('\n' + '═' * 70)
print('📄 REPORT GENERATION')
print('═' * 70)

# Calculate comprehensive KPIs
kpis = {
    'report_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_customers': int(len(df)),
    'churn_rate': float((df['Churn']=='Yes').mean()),
    'churned_customers': int((df['Churn']=='Yes').sum()),
    'active_customers': int((df['Churn']=='No').sum()),
    'total_revenue': float(df['TotalCharges'].sum()),
    'avg_monthly_charges': float(df['MonthlyCharges'].mean()),
    'avg_tenure_months': float(df['tenure'].mean()),
    'high_risk_customers': int(df[df['Contract']=='Month-to-month']['CustomerID'].count())
}

print('\n📊 Key Performance Indicators:')
for key, value in kpis.items():
    if key != 'report_date':
        if 'rate' in key:
            print(f'  {key}: {value:.2%}')
        elif 'revenue' in key or 'charges' in key:
            print(f'  {key}: ${value:,.2f}')
        elif 'months' in key:
            print(f'  {key}: {value:.1f}')
        else:
            print(f'  {key}: {value:,}')

In [ ]:
# Export to CSV
print('\n📊 Exporting to CSV...')

# Main dataset
df.to_csv('reports/csv/telecom_churn_complete.csv', index=False)
print('✅ Complete dataset: reports/csv/telecom_churn_complete.csv')

# KPIs
pd.DataFrame([kpis]).to_csv('reports/csv/kpis_summary.csv', index=False)
print('✅ KPIs summary: reports/csv/kpis_summary.csv')

# Churn analysis
churn_analysis = df.groupby('Contract').agg({
    'CustomerID': 'count',
    'Churn': lambda x: (x=='Yes').mean(),
    'MonthlyCharges': 'mean',
    'tenure': 'mean'
}).round(2)
churn_analysis.columns = ['Customers', 'ChurnRate', 'AvgCharges', 'AvgTenure']
churn_analysis.to_csv('reports/csv/churn_by_contract.csv')
print('✅ Churn analysis: reports/csv/churn_by_contract.csv')

In [ ]:
# Generate summary report (JSON)
summary_report = {
    'metadata': {
        'project': 'Telecom X - Churn Analysis',
        'generated_at': datetime.now().isoformat(),
        'version': '1.0.0',
        'author': 'Elizabeth Díaz Familia'
    },
    'kpis': kpis,
    'insights': [
        f"Analyzed {kpis['total_customers']:,} customer records",
        f"Overall churn rate: {kpis['churn_rate']:.2%}",
        f"Total revenue at risk: ${(kpis['churned_customers'] * kpis['avg_monthly_charges']):,.2f}/month",
        f"High-risk customers (Month-to-month): {kpis['high_risk_customers']:,}",
        f"Average customer tenure: {kpis['avg_tenure_months']:.1f} months"
    ],
    'recommendations': [
        'Focus retention efforts on month-to-month contract customers',
        'Implement loyalty programs for customers with low tenure',
        'Investigate reasons for fiber optic service churn',
        'Consider incentives for contract upgrades',
        'Monitor new customers closely during first 12 months'
    ]
}

with open('reports/executive_summary.json', 'w') as f:
    json.dump(summary_report, f, indent=4)

print('\n✅ Executive summary: reports/executive_summary.json')

## 🎉 FINAL SUMMARY

In [ ]:
print('\n' + '═' * 70)
print('🎉 ANALYSIS COMPLETED SUCCESSFULLY')
print('═' * 70)

print('\n✅ COMPLETED SECTIONS:')
print('  1️⃣ Setup & Configuration')
print('  2️⃣ ETL Pipeline (Extract, Transform, Load)')
print('  3️⃣ Exploratory Data Analysis')
print('  4️⃣ Data Visualization (Plotly)')
print('  5️⃣ Statistical Analysis')
print('  6️⃣ Machine Learning (K-Means Clustering)')
print('  7️⃣ Report Generation (CSV, JSON)')

print('\n📊 KEY FINDINGS:')
print(f'  • Total Customers: {kpis["total_customers"]:,}')
print(f'  • Churn Rate: {kpis["churn_rate"]:.2%}')
print(f'  • Total Revenue: ${kpis["total_revenue"]:,.2f}')
print(f'  • High-Risk Customers: {kpis["high_risk_customers"]:,}')

print('\n📁 GENERATED FILES:')
print('  📊 CSV Reports:')
print('     - reports/csv/telecom_churn_complete.csv')
print('     - reports/csv/kpis_summary.csv')
print('     - reports/csv/churn_by_contract.csv')
print('  📄 JSON Reports:')
print('     - reports/executive_summary.json')
print('  💾 Processed Data:')
print('     - data/processed/telecom_churn_clean.csv')

print('\n💡 RECOMMENDATIONS:')
for i, rec in enumerate(summary_report['recommendations'], 1):
    print(f'  {i}. {rec}')

print('\n' + '═' * 70)
print('✨ Thank you for using Telecom X Churn Analysis!')
print('👤 Developed by Elizabeth Díaz Familia')
print('🔗 GitHub: https://github.com/Lizzy0981')
print('═' * 70)